In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [7]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [9]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7])


In [11]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [13]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [15]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [17]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.0491, -0.5835, -0.3241, -0.0429, -0.1859, -0.0338,  0.3057,  0.0170,
         -0.5282, -0.1926, -0.3395, -0.1546,  0.3528,  0.3319, -0.3616, -0.2717,
         -0.6921,  0.1686,  0.1057,  0.1522],
        [ 0.0715, -0.4070, -0.1496,  0.3143,  0.2732, -0.3637,  0.2139, -0.2398,
         -0.2892, -0.2600, -0.0535, -0.0410,  0.3122,  0.2520, -0.3528, -0.2013,
         -0.5694,  0.2658,  0.1355,  0.1937],
        [ 0.0102, -0.7443, -0.1676,  0.3372, -0.0698, -0.1683,  0.2447,  0.1603,
         -0.3753, -0.2963,  0.0011,  0.1632,  0.3491,  0.3414, -0.0698, -0.4021,
         -0.7412,  0.1766,  0.3611,  0.3201]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0491, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3057, 0.0170, 0.0000,
         0.0000, 0.0000, 0.0000, 0.3528, 0.3319, 0.0000, 0.0000, 0.0000, 0.1686,
         0.1057, 0.1522],
        [0.0715, 0.0000, 0.0000, 0.3143, 0.2732, 0.0000, 0.2139, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.3122, 0.2520, 0.00

In [19]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [21]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [23]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0075,  0.0213, -0.0342,  ...,  0.0165,  0.0151, -0.0250],
        [ 0.0248,  0.0270,  0.0223,  ..., -0.0156,  0.0170, -0.0272]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0214, -0.0331], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0349, -0.0311, -0.0265,  ...,  0.0037, -0.0347,  0.0167],
        [ 0.0316, -0.0411,  0.0013,  ...,  0.0280,  0.0090, -0.0302]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 